In [846]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import math

# EDA

In [847]:
raw_path = '../data/raw'
train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [848]:
train

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,...,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,...,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,54-103-006-01,М,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,операторы и монтажники установок и машинного о...,1,0,...,06:00:00,0,0,0,54-103-006-01,0,0,0,0,0
951,54-701-039-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,07:15:00,0,0,0,54-701-039-01,0,0,0,0,0
952,54-002-014-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,служащие,0,1,...,08:00:00,1,0,0,54-002-014-01,1,0,1,0,0
953,54-501-022-01,Ж,в браке в настоящее время,европейская,Русские,Атеист / агностик,5 - ВУЗ,"представители законодат. органов власти,...",1,0,...,08:30:00,0,1,0,54-501-022-01,0,0,0,0,0


In [849]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
ID_COL = 'ID'
EDU_COL = 'Образование'
SEX_COL = 'Пол'
CAT_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
OHE_COLS = [
    'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког']

In [850]:
def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
    df = df.set_index(idx_col)
    return df

In [851]:
train, target = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]

In [852]:
def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
    if 'ID_y' in df.columns:
        df = df.drop('ID_y', axis=1)
    return df

In [853]:
def add_ord_edu(df: pd.DataFrame) -> pd.DataFrame:
    df[f'{EDU_COL}_ord'] = df[EDU_COL].str.slice(0, 1).astype(np.int8).values
    return df

In [854]:
def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
    most_freq = df[SEX_COL].value_counts().index[0]
    df[SEX_COL] = df[SEX_COL].fillna(most_freq)
    return df

In [855]:
def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df[CAT_COLS] = df[CAT_COLS].astype('category')

    ohe_int_cols = train[OHE_COLS].select_dtypes('number').columns
    df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

    df[REAL_COLS] = df[REAL_COLS].astype(np.float32)
    return df
    

In [856]:
def delete_nan_value(df: pd.DataFrame) ->pd.DataFrame:
    ##Smoking
    period_smoke = list(df['Возраст курения'])
    number_of_сigarettes = list(df['Сигарет в день'])
    count = 0
    for index, row in df.iterrows():
        if row['Статус Курения'] == 'Никогда не курил(а)':
            if pd.isna(row['Возраст курения']):
                period_smoke[count] = 0
            if pd.isna(row['Сигарет в день']):
                number_of_сigarettes[count] = 0
        count += 1
    df = df.drop(columns=['Возраст курения', 'Сигарет в день'])
    count = 0
    for index, row in df.iterrows():
        df.loc[index,'Возраст курения'] = period_smoke[count]
        df.loc[index,'Сигарет в день'] = number_of_сigarettes[count]
        count += 1 
    df = df[(np.isnan(df['Сигарет в день'])) == False] 

    ## Пассивное курение 
    frequency_second_hand_smoke = list(df['Частота пасс кур'])
    count = 0
    for index, row in df.iterrows():
        if row['Пассивное курение'] == 0:
            if pd.isna(row['Частота пасс кур']):
                frequency_second_hand_smoke[count] = 'менее 1 раза в день'
        count += 1
    df = df.drop(columns=['Частота пасс кур'])
    df.insert(24, 'Частота пасс кур', frequency_second_hand_smoke, True)
    df = df[(pd.isna(df['Частота пасс кур'])) == False] 
    
    ## алкоголь
    drinking_period = list(df['Возраст алког'])
    count = 0
    for index, row in df.iterrows():
        if row['Алкоголь'] == 'никогда не употреблял':
            if pd.isna(row['Возраст алког']):
                drinking_period[count] = 0
        count += 1
    df = df.drop(columns=['Возраст алког'])
    count = 0
    for index, row in df.iterrows():
        df.loc[index,'Возраст алког'] = drinking_period[count]
        count += 1
    df = df[(pd.isna(df['Возраст алког'])) == False] 
    return df

In [857]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = set_idx(df, ID_COL)
    df = drop_unnecesary_id(df)
    df = fill_sex(df)
    df = cast_types(df)
    df = add_ord_edu(df)
    df = delete_nan_value(df)
    return df

In [858]:
train = preprocess(train)
test = preprocess(test)

# Modeling